In [42]:
# 1: RUN AT EVERY NEW RUNTIME
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms

import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import seaborn as sns

import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

# Install fvcore
!pip install fvcore

In [41]:
# 2: NEED TO RUN AT AVERY NEW RUNTIME
# Setup google drive connection
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import gspread
from google.auth import default
from google.colab import auth

# Autenticazione
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

Mounted at /content/drive


In [ ]:
# TO BE RUNNED JUST THE FIRST TIME!
# Setup git integration
#%cd /content/drive/MyDrive
#!git clone https://github.com/s348174/MLDL2024_project1.git
#%cd

UsageError: Line magic function `%#cd` not found.


In [40]:
# 3. RUN EVERYTIME IN ORDER TO AVOID CONFLICTS
# Update drive git directory
%cd /content/drive/MyDrive/MLDL2024_project1
!git pull
%cd
# 4. RUN AGAIN POINT N.2

/content/drive/MyDrive/MLDL2024_project1
Already up to date.
/root


In [44]:
# 5. RUN THIS CELL TO TRAIN
import sys
import os
from tempfile import TemporaryDirectory

# Add the directory containing train.py to the system path
google_drive_path = '/content/drive/MyDrive/MLDL2024_project1'
sys.path.append(google_drive_path)
datasets_folder_path = '/content/drive/MyDrive/MLDL2024_project1/datasets'
sys.path.append(datasets_folder_path)
models_path = '/content/drive/MyDrive/MLDL2024_project1/models/deeplabv2'
sys.path.append(models_path)
from train import deeplab_train, deeplab_test

# Open zipfile
import zipfile
zip_path = '/content/drive/MyDrive/machine_learning_shared_2025/Cityscapes.zip'
workspace_path = google_drive_path
with zipfile.ZipFile(zip_path) as z:
  # Mostra i file contenuti
  #print("Contenuto dello ZIP:", z.namelist())
  with TemporaryDirectory() as tmpdir:
    z.extractall(tmpdir)
    #image_folder_path=tmpdir+"/Cityscapes/Cityspaces/images/train"
    #train_data = datasets.ImageFolder(
    #    root=image_folder_path,
    #    transform=ToTensor()
    #)
    #test_folder_path=tmpdir+"/Cityscapes/Cityspaces/images/test"
    #test_data = datasets.ImageFolder(
    #    root=image_folder_path,
    #    transform=ToTensor()
    #)

    # Creates symlink to data folder
    real_path = "tmpdir"
    #destinazione_symlink = "/tmp/tmpcp08lfus"
    #if not os.path.exists(destinazione_symlink):
    #    os.symlink(real_path, destinazione_symlink)

    num_epochs = 50
    batch_size = 4
    dataset_path = tmpdir+"/Cityscapes/Cityspaces"
    pretrained_path = workspace_path + "/deeplab_resnet_pretrained_imagenet.pth"
    deeplab_train(dataset_path, workspace_path, pretrained_path, num_epochs = num_epochs, batch_size = batch_size)
    model_path = workspace_path + "/export/deeplabv2_final.pth"
    deeplab_test(dataset_path, model_path)

TypeError: deeplab_train() takes from 3 to 4 positional arguments but 5 were given

In [ ]:
#to check core availability
import multiprocessing
max_num_workers = multiprocessing.cpu_count() #colab pro has 4 (the default has just 2) (for Emanuele)
print(f"Number of CPU cores: {max_num_workers}") #when Emanuele runs the code, it prints 8

Number of CPU cores: 12
